In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
from pyspark.sql import functions as F
from pyspark.sql.functions import mean
from pyspark.sql.functions import to_timestamp
import datetime

spark = (SparkSession.builder
        .appName("MySparkApp")
        .enableHiveSupport()
        .getOrCreate())

In [9]:
crd = (spark.read.option('header', 'true')
            .csv('hdfs://172.17.0.2:9000/ingest/automobiles/CarRentalData.csv'))
grf_usa = (spark.read.option('header', 'true')
                     .option('delimiter', ';')
                     .csv('hdfs://172.17.0.2:9000/ingest/automobiles/georef-united-states-of-america.csv'))

In [10]:
crd.columns

['fuelType',
 'rating',
 'renterTripsTaken',
 'reviewCount',
 'location.city',
 'location.country',
 'location.latitude',
 'location.longitude',
 'location.state',
 'owner.id',
 'rate.daily',
 'vehicle.make',
 'vehicle.model',
 'vehicle.type',
 'vehicle.year']

In [11]:
# Rename Car Rental Data column names

crd = (crd.withColumnRenamed('location.city', 'city')
          .withColumnRenamed('location.country', 'country')
          .withColumnRenamed('location.latitude', 'latitude')
          .withColumnRenamed('location.longitude', 'longitude')
          .withColumnRenamed('location.state', 'state')
          .withColumnRenamed('owner.id', 'owner_id')
          .withColumnRenamed('rate.daily', 'rate_daily')
          .withColumnRenamed('vehicle.make', 'make')
          .withColumnRenamed('vehicle.model', 'model')
          .withColumnRenamed('vehicle.type', 'type')
          .withColumnRenamed('vehicle.year', 'year'))
                

In [12]:
# Cast Car Rental Data columns
crd = (crd.withColumn('renterTripsTaken', F.col('renterTripsTaken').cast(IntegerType()))
          .withColumn('reviewCount', F.col('reviewCount').cast(IntegerType()))
          .withColumn('owner_id', F.col('owner_id').cast(IntegerType()))
          .withColumn('rate_daily', F.col('rate_daily').cast(IntegerType()))
          .withColumn('year', F.col('year').cast(IntegerType())))

In [18]:
grf_usa = (grf_usa.withColumnRenamed('Geo Point', 'geo_point')
                  .withColumnRenamed('Geo Shape', 'geo_shape')
                  .withColumnRenamed('Official Code State', 'OCS')
                  .withColumnRenamed('Official Name State', 'ONS')
                  .withColumnRenamed('Iso 3166-3 Area Code', 'iso_3166_3_ac')
                  .withColumnRenamed('Type', 'type')
                  .withColumnRenamed('United States Postal Service state abbreviation', 'state')
                  .withColumnRenamed('State FIPS Code', 'FIPS')
                  .withColumnRenamed('State GNIS Code', 'GNIS')
                  .withColumnRenamed('Year', 'usa_year'))
grf = (grf_usa.withColumn('usa_year', F.col('usa_year').cast(IntegerType()))
              .withColumn('OCS', F.col('OCS').cast(IntegerType())))

In [6]:
df = crd.join(grf_usa, on='state') 
df = df.na.drop(subset=['rating'])
df = df.withColumn('fuelType', F.lower('fuelType'))
df = df.filter(df.state != 'TX')

In [7]:
crd.printSchema()

root
 |-- fuelType: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- renterTripsTaken: integer (nullable = true)
 |-- reviewCount: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- state: string (nullable = true)
 |-- owner_id: integer (nullable = true)
 |-- rate_daily: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)



# Problema 5
Por medio de consultas SQL al data-warehouse, mostrar

a. Cantidad de alquileres de autos, teniendo en cuenta sólo los vehículos
ecológicos (fuelType hibrido o eléctrico) y con un rating de al menos 4.

In [8]:
ecologic = ['electric', 'hybrid']
rent_ecologic = df.filter((F.col('fuelType').isin(ecologic)) & (df.rating >= 4))
print ("Cantidad de alquileres de autos ecológicos con rating >= 4:", rent_ecologic.count())


Cantidad de alquileres de autos ecológicos con rating >= 4 770


b. Los 5 estados con menor cantidad de alquileres (mostrar query y visualización).

In [9]:
df.groupBy('state').count().orderBy('count').show(10)

[Stage 7:>                                                          (0 + 1) / 1]

+-----+-----+
|state|count|
+-----+-----+
|   MT|    1|
|   WV|    3|
|   NH|    3|
|   DE|    4|
|   AR|    4|
|   MS|    4|
|   IA|   11|
|   DC|   11|
|   ME|   11|
|   NE|   14|
+-----+-----+
only showing top 10 rows



c. Los 10 modelos (junto con su marca) de autos más rentados (mostrar query y
visualización).

In [10]:
df.groupBy('model', 'make').count().orderBy('count', ascending=False).show(10)

+--------+-------------+-----+
|   model|         make|count|
+--------+-------------+-----+
| Model 3|        Tesla|  288|
| Mustang|         Ford|  136|
| Model S|        Tesla|  122|
|Wrangler|         Jeep|  108|
| Model X|        Tesla|  103|
| C-Class|Mercedes-Benz|   78|
| Corolla|       Toyota|   78|
|3 Series|          BMW|   76|
|Corvette|    Chevrolet|   68|
|  Camaro|    Chevrolet|   61|
+--------+-------------+-----+
only showing top 10 rows



d. Mostrar por año, cuántos alquileres se hicieron, teniendo en cuenta automóviles
fabricados desde 2010 a 2015.

In [11]:
rent_year = (crd.filter((crd.year >= '2010') & (crd.year <= '2015'))
               .groupBy('year')
               .count()
               .orderBy('year'))
rent_year.show()

+----+-----+
|year|count|
+----+-----+
|2010|  166|
|2011|  238|
|2012|  291|
|2013|  371|
|2014|  453|
|2015|  627|
+----+-----+



e. las 5 ciudades con más alquileres de vehículos ecológicos (fuelType hibrido o
electrico)

In [12]:
ecologic = ['hybrid', 'electric']
(df.filter((F.col('fuelType').isin(ecologic)))
          .groupBy('city')
          .count()
          .orderBy('count', ascending=False)).show(5)

+---------+-----+
|     city|count|
+---------+-----+
|San Diego|   44|
|Las Vegas|   34|
| Portland|   20|
|  Phoenix|   17|
| San Jose|   15|
+---------+-----+
only showing top 5 rows



f. El promedio de reviews, segmentando por tipo de combustible.

In [15]:
mean_reviews = df.groupBy('fuelType').avg('reviewCount').orderBy('avg(reviewCount)', ascending=False)
mean_reviews.show()

+--------+------------------+
|fuelType|  avg(reviewCount)|
+--------+------------------+
|  hybrid| 34.87336244541485|
|gasoline|31.927023661270237|
|electric|28.339483394833948|
|    null|21.049180327868854|
|  diesel|              17.5|
+--------+------------------+



# 6 Conclusiones del proyecto
Este proyecto es interesante porque estudia la cantidad de alquileres de autos por ciudad, y sobre todo la cantidad de autos ecológicos alquilados. Vemos que 
- Montana es el estado con menor cantidad de alquileres
- Model 3, Tesla fue el vehículo más alquilado
- el número de alquileres creció desde los vehículos fabricados en 2010 a los fabricados en el 2015
- San Diego es la ciudad con mayor cantidad de autos ecológicos alquilados
- el máximo promedio de review se dio para los autos híbridos, y el menor promedio para los vehículos propulsados con motor diesel

Agregaría a este dataset la fecha en la que cada auto fue alquilado. De este modo, se podría estudiar tanto la evolución temporal de autos alquilados, como la progresión del tipo de combustible de los vehículos, por ejemplo.

# 7 - Arquitectura alternativa

Agregaría al datawarehouse las columnas:
- id_rent
- date_rent
- time_rent